# Training Notebook

## Validate Your Training, Validating and Running Data being Formatted Properly with file 1_DataProcess.ipynb

### Place your training augments at ./experiment/TRAIN_TEST.yaml or change it at the Paramenters configuration section 

```
# -------training data information-------
train_data_dir: ./dataset/train # training embryo folder
train_embryos: [170614plc1p1, 170704plc1p1]  # embryos used for training.
max_times: [100,140]
# network/model information
net: MDFNetBin #  Parameters used for training.
net_params:
  in_channels: 1
  n_first: 32
  conv_channels: 64
  groups: 16
  norm: in #sync_bn
  out_class: 1
criterion: mse_loss
criterion_kwargs:
  weight_type: square
# ------data loader name-------
dataset: Memb3DDataset
seed: 1024
batch_size: 2
num_epochs: 50    # small model, quick computing, less epochs
save_freq: 10     # save every 50 epochs
valid_freq: 10   # validate every 10 epochs
start_iter: 0
# ------optimizaiton parameters-----
opt: Adam
opt_params:
  lr: 5e-3
  weight_decay: 1e-5
  amsgrad: true
workers: 8
# ----volume transformation, data augmentation-----
train_transforms: 
  Compose([
    ContourEDT(9),
    RandomIntensityChange([0.1, 0.1]),
    RandCrop((128,128,128)),
    RandomFlip(0),
    NumpyType((np.float32, np.float32, np.float32, np.float32))
    ])

```

#### Import dependency library

In [1]:
import os
import time
import logging
import random
import argparse
import setproctitle
import numpy as np

#  torch library
import torch
import torch.optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader

#  user defined library
import models 
from utils import ParserUse, criterions
from utils.show_train import Visualizer
from utils.train_utils import adjust_learning_rate
#  datasets
from data import datasets
from data.data_utils import init_fn
from data.sampler import CycleSampler


from utils.criterions import mse_loss, generalized_dice_loss

#### Paramenters configuration section

In [2]:
cudnn.benchmark = True  # let auto-tuner find the most efficient network (used in fixed network.)

## parse arguments
args = ParserUse('TRAIN_TEST', log='train')
# CHANGE Your training parameters here
args.gpu='0'
args.show_image_freq=3
args.show_loss_freq=3

cpkts = args.makedir()


In [3]:
# set up visdom to view the training situation
if args.show_image_freq > 0 or args.show_loss_freq > 0:
    visualizer = Visualizer(1)

2023-07-13 17:39:27,405 Setting up a new session...
2023-07-13 17:39:27,413 Websocket connected
2023-07-13 17:39:27,414 Visdom successfully connected to server


In [4]:
# set cuda, set gpu in usage
os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu

# =====================================================
#   set seed for randomization
# =====================================================
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)

# =====================================================
#  construct network
# =====================================================
Network = getattr(models, args.net)
model = Network(**args.net_params)
model = torch.nn.DataParallel(model).cuda()
optimizer = getattr(torch.optim, args.opt)(model.parameters(), **args.opt_params)
criterion = getattr(criterions, args.criterion)

#### Set up dataset, dataloader

In [5]:
# =====================================================
#  set dataset loader
# =====================================================
Dataset = getattr(datasets, args.dataset)
print(Dataset)
train_set = Dataset(root=args.train_data_dir, membrane_names=args.train_embryos, for_train=True,
                    transforms=args.train_transforms, return_target=True, max_times=args.train_max_times)
num_iters = args.num_iters or (len(train_set) * args.num_epochs) // args.batch_size
num_iters -= args.start_iter
train_sampler = CycleSampler(len(train_set), num_iters * args.batch_size)
train_loader = DataLoader(
    dataset=train_set,
    batch_size=args.batch_size,
    # collate_fn=train_set.collate,
    sampler=train_sampler,
    num_workers=args.workers,
    pin_memory=True,
    worker_init_fn=init_fn
)

<class 'data.datasets.Memb3DDataset'>


#### Start go through dataloader and training

In [9]:
# ====================================================
#  start training
# ====================================================
torch.set_grad_enabled(True)
enum_batches = len(train_set) / float(args.batch_size)  # number of epoches in each iteration
for i, data in enumerate(train_loader, args.start_iter):
    #  record process
    elapsed_bsize = int(i / enum_batches) + 1
    epoch = int((i + 1) / enum_batches)
    setproctitle.setproctitle("Epoch: {}/{}".format(elapsed_bsize, args.num_epochs))  # set process name

    #  adjust learning super parameters
    adjust_learning_rate(optimizer, epoch, args.num_epochs, args.opt_params.lr)

    #  go through the network
    data = [t.cuda(non_blocking=True) for t in data]  # Set non_blocking for multiple GPUs
    raw, seg_nuc, target_edt = data[:3]
    prediction = model(raw)


    loss = criterion(prediction, target_edt,)

    #  backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # =============================================================
    #   Show mmiddle results
    if args.show_image_freq > 0 and (i % args.show_image_freq) == 0:
        image_dict = dict(Raw=raw[0, 0, :, :, 31].float(), target_edt=target_edt[0, 0, :, :, 31].float(),
                          prediction=prediction[0, 0, :, :, 31].float())
        visualizer.show_current_images(image_dict)
    if args.show_loss_freq > 0 and (i % args.show_loss_freq) == 0:
        visualizer.plot_current_losses(progress_ratio=(i + 1) / enum_batches,
                                       losses=dict(loss=loss.item()))
    # =============================================================

    #  save trained model
    if (i + 1) % int(enum_batches * args.save_freq) == 0:
        file_name = os.path.join(cpkts, "model_epoch_{}.pth".format(epoch))
        torch.save(dict(
            iter=i,
            state_dict=model.state_dict(),
            optim_dict=optimizer.state_dict()
        ), file_name)

    logging.info("Iter {0:}, Epoch {1:.4f}, Loss {2:.7f}".format(i + 1, (i + 1) / enum_batches, loss.item()))


2023-07-13 18:01:22,788 Iter 1, Epoch 0.0185, Loss 0.0061434
2023-07-13 18:01:22,963 Iter 2, Epoch 0.0370, Loss 0.0039793
2023-07-13 18:01:23,137 Iter 3, Epoch 0.0556, Loss 0.0036540
2023-07-13 18:01:23,332 Iter 4, Epoch 0.0741, Loss 0.0024059
2023-07-13 18:01:23,506 Iter 5, Epoch 0.0926, Loss 0.0031705
2023-07-13 18:01:23,681 Iter 6, Epoch 0.1111, Loss 0.0049364
2023-07-13 18:01:23,875 Iter 7, Epoch 0.1296, Loss 0.0025266
2023-07-13 18:01:24,049 Iter 8, Epoch 0.1481, Loss 0.0030266
2023-07-13 18:01:42,559 Iter 9, Epoch 0.1667, Loss 0.0039793
2023-07-13 18:01:42,765 Iter 10, Epoch 0.1852, Loss 0.0054371
2023-07-13 18:01:43,472 Iter 11, Epoch 0.2037, Loss 0.0051526
2023-07-13 18:01:43,646 Iter 12, Epoch 0.2222, Loss 0.0041327
2023-07-13 18:01:43,840 Iter 13, Epoch 0.2407, Loss 0.0032204
2023-07-13 18:01:44,015 Iter 14, Epoch 0.2593, Loss 0.0030735
2023-07-13 18:01:44,190 Iter 15, Epoch 0.2778, Loss 0.0070291
2023-07-13 18:01:44,385 Iter 16, Epoch 0.2963, Loss 0.0028777
2023-07-13 18:02:

2023-07-13 18:06:39,234 Iter 133, Epoch 2.4630, Loss 0.0048476
2023-07-13 18:06:39,409 Iter 134, Epoch 2.4815, Loss 0.0064282
2023-07-13 18:06:39,585 Iter 135, Epoch 2.5000, Loss 0.0033817
2023-07-13 18:06:39,780 Iter 136, Epoch 2.5185, Loss 0.0029970
2023-07-13 18:06:58,406 Iter 137, Epoch 2.5370, Loss 0.0023928
2023-07-13 18:06:58,585 Iter 138, Epoch 2.5556, Loss 0.0046165
2023-07-13 18:06:58,783 Iter 139, Epoch 2.5741, Loss 0.0063947
2023-07-13 18:06:58,960 Iter 140, Epoch 2.5926, Loss 0.0022629
2023-07-13 18:06:59,134 Iter 141, Epoch 2.6111, Loss 0.0019480
2023-07-13 18:06:59,331 Iter 142, Epoch 2.6296, Loss 0.0031052
2023-07-13 18:06:59,507 Iter 143, Epoch 2.6481, Loss 0.0087079
2023-07-13 18:06:59,682 Iter 144, Epoch 2.6667, Loss 0.0025849
2023-07-13 18:07:17,796 Iter 145, Epoch 2.6852, Loss 0.0015953
2023-07-13 18:07:17,971 Iter 146, Epoch 2.7037, Loss 0.0022522
2023-07-13 18:07:18,146 Iter 147, Epoch 2.7222, Loss 0.0024564
2023-07-13 18:07:18,339 Iter 148, Epoch 2.7407, Loss 0.

2023-07-13 18:11:49,008 Iter 264, Epoch 4.8889, Loss 0.0020156
2023-07-13 18:12:01,535 Iter 265, Epoch 4.9074, Loss 0.0028872
2023-07-13 18:12:01,720 Iter 266, Epoch 4.9259, Loss 0.0023334
2023-07-13 18:12:01,895 Iter 267, Epoch 4.9444, Loss 0.0020612
2023-07-13 18:12:07,262 Iter 268, Epoch 4.9630, Loss 0.0017178
2023-07-13 18:12:07,437 Iter 269, Epoch 4.9815, Loss 0.0024897
2023-07-13 18:12:07,612 Iter 270, Epoch 5.0000, Loss 0.0018450
2023-07-13 18:12:07,809 Iter 271, Epoch 5.0185, Loss 0.0017270
2023-07-13 18:12:07,985 Iter 272, Epoch 5.0370, Loss 0.0020098
2023-07-13 18:12:20,210 Iter 273, Epoch 5.0556, Loss 0.0022937
2023-07-13 18:12:20,415 Iter 274, Epoch 5.0741, Loss 0.0016148
2023-07-13 18:12:20,590 Iter 275, Epoch 5.0926, Loss 0.0032840
2023-07-13 18:12:26,124 Iter 276, Epoch 5.1111, Loss 0.0024389
2023-07-13 18:12:26,321 Iter 277, Epoch 5.1296, Loss 0.0014085
2023-07-13 18:12:26,497 Iter 278, Epoch 5.1481, Loss 0.0019489
2023-07-13 18:12:26,673 Iter 279, Epoch 5.1667, Loss 0.

2023-07-13 18:17:05,437 Iter 395, Epoch 7.3148, Loss 0.0045408
2023-07-13 18:17:05,612 Iter 396, Epoch 7.3333, Loss 0.0032127
2023-07-13 18:17:05,808 Iter 397, Epoch 7.3519, Loss 0.0022558
2023-07-13 18:17:05,984 Iter 398, Epoch 7.3704, Loss 0.0016738
2023-07-13 18:17:06,159 Iter 399, Epoch 7.3889, Loss 0.0076844
2023-07-13 18:17:06,351 Iter 400, Epoch 7.4074, Loss 0.0023243
2023-07-13 18:17:23,971 Iter 401, Epoch 7.4259, Loss 0.0071314
2023-07-13 18:17:24,146 Iter 402, Epoch 7.4444, Loss 0.0017748
2023-07-13 18:17:24,342 Iter 403, Epoch 7.4630, Loss 0.0023977
2023-07-13 18:17:24,517 Iter 404, Epoch 7.4815, Loss 0.0041561
2023-07-13 18:17:24,693 Iter 405, Epoch 7.5000, Loss 0.0026246
2023-07-13 18:17:24,888 Iter 406, Epoch 7.5185, Loss 0.0018564
2023-07-13 18:17:25,064 Iter 407, Epoch 7.5370, Loss 0.0031542
2023-07-13 18:17:25,240 Iter 408, Epoch 7.5556, Loss 0.0017762
2023-07-13 18:17:43,684 Iter 409, Epoch 7.5741, Loss 0.0020560
2023-07-13 18:17:43,859 Iter 410, Epoch 7.5926, Loss 0.

2023-07-13 18:22:07,952 Iter 526, Epoch 9.7407, Loss 0.0014791
2023-07-13 18:22:08,127 Iter 527, Epoch 9.7593, Loss 0.0010892
2023-07-13 18:22:08,302 Iter 528, Epoch 9.7778, Loss 0.0019901
2023-07-13 18:22:26,211 Iter 529, Epoch 9.7963, Loss 0.0008802
2023-07-13 18:22:26,395 Iter 530, Epoch 9.8148, Loss 0.0022160
2023-07-13 18:22:26,569 Iter 531, Epoch 9.8333, Loss 0.0013989
2023-07-13 18:22:26,767 Iter 532, Epoch 9.8519, Loss 0.0019788
2023-07-13 18:22:26,943 Iter 533, Epoch 9.8704, Loss 0.0014507
2023-07-13 18:22:27,118 Iter 534, Epoch 9.8889, Loss 0.0009594
2023-07-13 18:22:27,315 Iter 535, Epoch 9.9074, Loss 0.0014383
2023-07-13 18:22:27,490 Iter 536, Epoch 9.9259, Loss 0.0022817
2023-07-13 18:22:44,732 Iter 537, Epoch 9.9444, Loss 0.0015075
2023-07-13 18:22:44,927 Iter 538, Epoch 9.9630, Loss 0.0020867
2023-07-13 18:22:45,102 Iter 539, Epoch 9.9815, Loss 0.0037071
2023-07-13 18:22:45,326 Iter 540, Epoch 10.0000, Loss 0.0017898
2023-07-13 18:22:46,581 Iter 541, Epoch 10.0185, Loss 

...

2023-07-13 19:27:35,594 Iter 2184, Epoch 40.4444, Loss 0.0008327
2023-07-13 19:27:35,796 Iter 2185, Epoch 40.4630, Loss 0.0012166
2023-07-13 19:27:35,972 Iter 2186, Epoch 40.4815, Loss 0.0021873
2023-07-13 19:27:38,150 Iter 2187, Epoch 40.5000, Loss 0.0007917
2023-07-13 19:27:38,357 Iter 2188, Epoch 40.5185, Loss 0.0008801
2023-07-13 19:27:52,490 Iter 2189, Epoch 40.5370, Loss 0.0009527
2023-07-13 19:27:52,665 Iter 2190, Epoch 40.5556, Loss 0.0006065
2023-07-13 19:27:52,864 Iter 2191, Epoch 40.5741, Loss 0.0011098
2023-07-13 19:27:54,572 Iter 2192, Epoch 40.5926, Loss 0.0010368
2023-07-13 19:27:54,748 Iter 2193, Epoch 40.6111, Loss 0.0006151
2023-07-13 19:27:54,948 Iter 2194, Epoch 40.6296, Loss 0.0013649
2023-07-13 19:27:56,603 Iter 2195, Epoch 40.6481, Loss 0.0017371
2023-07-13 19:27:56,778 Iter 2196, Epoch 40.6667, Loss 0.0003888
2023-07-13 19:28:11,311 Iter 2197, Epoch 40.6852, Loss 0.0015908
2023-07-13 19:28:11,487 Iter 2198, Epoch 40.7037, Loss 0.0020924
2023-07-13 19:28:11,662 I

2023-07-13 19:32:38,832 Iter 2311, Epoch 42.7963, Loss 0.0008088
2023-07-13 19:32:39,007 Iter 2312, Epoch 42.8148, Loss 0.0014291
2023-07-13 19:32:39,182 Iter 2313, Epoch 42.8333, Loss 0.0010370
2023-07-13 19:32:39,381 Iter 2314, Epoch 42.8519, Loss 0.0012463
2023-07-13 19:32:44,361 Iter 2315, Epoch 42.8704, Loss 0.0005303
2023-07-13 19:32:44,544 Iter 2316, Epoch 42.8889, Loss 0.0006526
2023-07-13 19:32:56,898 Iter 2317, Epoch 42.9074, Loss 0.0005538
2023-07-13 19:32:57,073 Iter 2318, Epoch 42.9259, Loss 0.0011057
2023-07-13 19:32:57,248 Iter 2319, Epoch 42.9444, Loss 0.0009187
2023-07-13 19:32:57,446 Iter 2320, Epoch 42.9630, Loss 0.0009112
2023-07-13 19:32:57,621 Iter 2321, Epoch 42.9815, Loss 0.0011447
2023-07-13 19:32:57,797 Iter 2322, Epoch 43.0000, Loss 0.0010000
2023-07-13 19:33:03,033 Iter 2323, Epoch 43.0185, Loss 0.0009351
2023-07-13 19:33:03,215 Iter 2324, Epoch 43.0370, Loss 0.0011762
2023-07-13 19:33:15,562 Iter 2325, Epoch 43.0556, Loss 0.0014367
2023-07-13 19:33:15,766 I

2023-07-13 19:37:42,045 Iter 2438, Epoch 45.1481, Loss 0.0012325
2023-07-13 19:37:42,220 Iter 2439, Epoch 45.1667, Loss 0.0004680
2023-07-13 19:37:43,995 Iter 2440, Epoch 45.1852, Loss 0.0008561
2023-07-13 19:37:44,170 Iter 2441, Epoch 45.2037, Loss 0.0011711
2023-07-13 19:37:44,346 Iter 2442, Epoch 45.2222, Loss 0.0014492
2023-07-13 19:37:44,545 Iter 2443, Epoch 45.2407, Loss 0.0004484
2023-07-13 19:37:44,721 Iter 2444, Epoch 45.2593, Loss 0.0003855
2023-07-13 19:38:00,849 Iter 2445, Epoch 45.2778, Loss 0.0008222
2023-07-13 19:38:01,058 Iter 2446, Epoch 45.2963, Loss 0.0009199
2023-07-13 19:38:01,233 Iter 2447, Epoch 45.3148, Loss 0.0009606
2023-07-13 19:38:02,758 Iter 2448, Epoch 45.3333, Loss 0.0006184
2023-07-13 19:38:02,958 Iter 2449, Epoch 45.3519, Loss 0.0014109
2023-07-13 19:38:03,134 Iter 2450, Epoch 45.3704, Loss 0.0018269
2023-07-13 19:38:03,309 Iter 2451, Epoch 45.3889, Loss 0.0004888
2023-07-13 19:38:03,506 Iter 2452, Epoch 45.4074, Loss 0.0005679
2023-07-13 19:38:20,344 I

2023-07-13 19:42:41,271 Iter 2565, Epoch 47.5000, Loss 0.0006345
2023-07-13 19:42:41,472 Iter 2566, Epoch 47.5185, Loss 0.0010455
2023-07-13 19:42:41,648 Iter 2567, Epoch 47.5370, Loss 0.0007993
2023-07-13 19:42:45,750 Iter 2568, Epoch 47.5556, Loss 0.0003802
2023-07-13 19:42:45,951 Iter 2569, Epoch 47.5741, Loss 0.0010845
2023-07-13 19:42:46,127 Iter 2570, Epoch 47.5926, Loss 0.0011026
2023-07-13 19:42:46,303 Iter 2571, Epoch 47.6111, Loss 0.0010335
2023-07-13 19:42:46,502 Iter 2572, Epoch 47.6296, Loss 0.0007831
2023-07-13 19:43:00,318 Iter 2573, Epoch 47.6481, Loss 0.0006204
2023-07-13 19:43:00,496 Iter 2574, Epoch 47.6667, Loss 0.0003669
2023-07-13 19:43:00,696 Iter 2575, Epoch 47.6852, Loss 0.0008140
2023-07-13 19:43:04,975 Iter 2576, Epoch 47.7037, Loss 0.0003667
2023-07-13 19:43:05,151 Iter 2577, Epoch 47.7222, Loss 0.0005840
2023-07-13 19:43:05,351 Iter 2578, Epoch 47.7407, Loss 0.0006899
2023-07-13 19:43:05,527 Iter 2579, Epoch 47.7593, Loss 0.0005831
2023-07-13 19:43:05,703 I

2023-07-13 19:47:32,168 Iter 2692, Epoch 49.8519, Loss 0.0004411
2023-07-13 19:47:36,782 Iter 2693, Epoch 49.8704, Loss 0.0006278
2023-07-13 19:47:36,966 Iter 2694, Epoch 49.8889, Loss 0.0008503
2023-07-13 19:47:37,166 Iter 2695, Epoch 49.9074, Loss 0.0005648
2023-07-13 19:47:37,974 Iter 2696, Epoch 49.9259, Loss 0.0002974
2023-07-13 19:47:38,150 Iter 2697, Epoch 49.9444, Loss 0.0008212
2023-07-13 19:47:38,350 Iter 2698, Epoch 49.9630, Loss 0.0009443
2023-07-13 19:47:38,526 Iter 2699, Epoch 49.9815, Loss 0.0016934
2023-07-13 19:47:38,756 Iter 2700, Epoch 50.0000, Loss 0.0003210


## From now, we have finished the training and got the model ./ckpts/TRAIN_TEST/model_epoch_50.pth

* This model pth is named as ./ckpts/{configuration yml file name}/model_epoch_{epoch numer}.pth
* You can add the epoch number and build your own yml file at **folder experiment**, for example, VNet_TRAIN_TEST.yml